In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from fastai import *
from fastai.vision import *
from fastai.callbacks import *

np.random.seed(42)
torch.cuda.set_device(1)

from annoy import AnnoyIndex

import glob

In [2]:
import requests

import psycopg2
import sys, os
import config as creds

In [3]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)


## Building Inventory Table

In [4]:
def connect():
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
    +" password="+ creds.PGPASSWORD
    conn=psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    return conn, cursor

In [5]:
def disconnect(conn, cursor):
    cursor.close()
    conn.close()
    print("Connection Closed!")
    

In [6]:
# Creating the inventory Table.  
# Should only need to create the table once

# Connecting to DB
conn, cursor = connect()

# SQL command to create inventory table
create_table = """
    CREATE TABLE IF NOT EXISTS inventory(
        index INTEGER,
        id TEXT PRIMARY KEY NOT NULL,
        category TEXT,
        image TEXT,
        displayName TEXT,
        urlHistory TEXT
    )
    """

# Execute SQL Command and commit to DB
cursor.execute(create_table)
conn.commit()

# Disconnect from DB
disconnect(conn, cursor)

Connected!
Connection Closed!


In [14]:
# Makes API calls to Rent the Runway to get inventory details for a given category of clothing
def get_category(category):
    has_next_page = True
    current_page = 1
    global index
    
    if category == 'dresses':
        url = 'https://www.renttherunway.com/c/'
    else:
        url = 'https://www.renttherunway.com/products/clothing/'
    
    conn, cursor = connect()
    
    while has_next_page:
        response = requests.get(url+category,
            params = {'filters[zip_code]': '10010','page':current_page},
            headers={'accept': 'application/json, text/javascript, */*; q=0.01', 'x-requested-with': 'XMLHttpRequest'})
        json_response = response.json()
        products = json_response['products']
        total_pages = json_response['totalPages']
        next_page = json_response['next_page']
        
        
        for product in products:
            
            try:
                product['images']['front']
                
                cursor.execute("INSERT INTO inventory (index,id,category,image,displayName,urlHistory) VALUES(%s,%s,%s,%s,%s,%s)",
                    (index,product['id'],category,product['images']['front']['270x'],product['displayName'],product['urlHistory'][-1]))
                
                index += 1
                
            except:
                continue
            
        
        conn.commit()
        
        
        print(category)
        print(f'Current page: {current_page}')
        print(f'Number of products collected: {len(products)}')
        print(f'Pages left: {total_pages-current_page}')
        print(index)
        
        if next_page:
            has_next_page = True
            current_page += 1
        else:
            has_next_page = False
            
    disconnect(conn, cursor)
        



In [15]:
#RTR Clothing Categories
categories = ['top','bottom','dresses','jumpsuit_romper','knit','jacket_coat']
#Want DB Index to start at 0
index = 0

for category in categories:
    get_category(category)

Connected!
top
Current page: 1
Number of products collected: 60
Pages left: 35
60
top
Current page: 2
Number of products collected: 60
Pages left: 34
120
top
Current page: 3
Number of products collected: 60
Pages left: 33
180
top
Current page: 4
Number of products collected: 60
Pages left: 32
240
top
Current page: 5
Number of products collected: 60
Pages left: 31
300
top
Current page: 6
Number of products collected: 60
Pages left: 30
360
top
Current page: 7
Number of products collected: 60
Pages left: 29
420
top
Current page: 8
Number of products collected: 60
Pages left: 28
480
top
Current page: 9
Number of products collected: 60
Pages left: 27
540
top
Current page: 10
Number of products collected: 60
Pages left: 26
600
top
Current page: 11
Number of products collected: 60
Pages left: 25
660
top
Current page: 12
Number of products collected: 60
Pages left: 24
720
top
Current page: 13
Number of products collected: 60
Pages left: 23
780
top
Current page: 14
Number of products collected:

dresses
Current page: 50
Number of products collected: 60
Pages left: 74
6399
dresses
Current page: 51
Number of products collected: 60
Pages left: 73
6459
dresses
Current page: 52
Number of products collected: 60
Pages left: 72
6519
dresses
Current page: 53
Number of products collected: 60
Pages left: 71
6579
dresses
Current page: 54
Number of products collected: 60
Pages left: 70
6639
dresses
Current page: 55
Number of products collected: 60
Pages left: 69
6699
dresses
Current page: 56
Number of products collected: 60
Pages left: 68
6759
dresses
Current page: 57
Number of products collected: 60
Pages left: 67
6819
dresses
Current page: 58
Number of products collected: 60
Pages left: 66
6879
dresses
Current page: 59
Number of products collected: 60
Pages left: 65
6939
dresses
Current page: 60
Number of products collected: 60
Pages left: 64
6999
dresses
Current page: 61
Number of products collected: 60
Pages left: 63
7059
dresses
Current page: 62
Number of products collected: 60
Pages 

knit
Current page: 18
Number of products collected: 19
Pages left: 0
12512
Connection Closed!
Connected!
jacket_coat
Current page: 1
Number of products collected: 60
Pages left: 19
12572
jacket_coat
Current page: 2
Number of products collected: 60
Pages left: 18
12632
jacket_coat
Current page: 3
Number of products collected: 60
Pages left: 17
12692
jacket_coat
Current page: 4
Number of products collected: 60
Pages left: 16
12752
jacket_coat
Current page: 5
Number of products collected: 60
Pages left: 15
12812
jacket_coat
Current page: 6
Number of products collected: 60
Pages left: 14
12872
jacket_coat
Current page: 7
Number of products collected: 60
Pages left: 13
12932
jacket_coat
Current page: 8
Number of products collected: 60
Pages left: 12
12992
jacket_coat
Current page: 9
Number of products collected: 60
Pages left: 11
13052
jacket_coat
Current page: 10
Number of products collected: 60
Pages left: 10
13112
jacket_coat
Current page: 11
Number of products collected: 60
Pages left: 